# * Do this only if you are using Google Colab. *

In [2]:
!mkdir -p ~/.kaggle

!cp kaggle.json /root/.kaggle/

import os
for filename in os.listdir('/content'):
    print(filename)

# Set the permissions for the kaggle.json file to make it readable only by the owner
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d hugomathien/soccer

.config
kaggle.json
sample_data
 15% 5.00M/32.7M [00:00<00:00, 39.3MB/s]
100% 32.7M/32.7M [00:00<00:00, 147MB/s] 


In [3]:
!unzip soccer.zip

Archive:  soccer.zip
  inflating: database.sqlite         


# 1. Importing libraries

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt

#Insert path here
database = '/content/database.sqlite'


conn = sqlite3.connect('/content/database.sqlite')

# 2. What's in the tables?

Lets start by seeing what is in the tables within our database, which we can get with the below query. Since we are actually running SQL queries in a python notebook, we need to write our SQL statements with a python wrapper around them. The key aspect is to place the SQL statement within each side of three speech marks.

In [5]:
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


So, as explained in the introduction, we have tables containing information about players, teams and matches.


**List of matches and goals in the English Premier League:**

*What are the matches played in the English Premier League during the 2015/16 season, and what were the scores?*

To answer this question, we need to join the match table with the country, league, and team tables twice. We rename some of the fields in the query results to avoid confusion, and we only select the fields that are relevant to our question.

The match table is a fact table, which contains all the facts or metrics or numbers about football matches. The country, league, and team tables are dimension tables, which contain information that people would normally like to filter on or summarize by.

In a live database, the match table would be updated every time there is a match. The dimension tables would only need to be updated much less often, for example at the start of a new football season.

The query results show the home team, away team, and score for each match in the English Premier League during the 2015/16 season.

In [6]:
detailed_matches = pd.read_sql("""
SELECT Match.id, Country.name AS country_name, League.name AS league_name,
       season, date, HT.team_long_name AS  home_team,
       AT.team_long_name AS away_team,
       home_team_goal, away_team_goal
FROM Match
LEFT JOIN Country on Country.id = Match.country_id
LEFT JOIN League on League.id = Match.league_id
LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
WHERE country_name = 'England' AND season = '2015/2016'
ORDER by date
LIMIT 10;""", conn)

detailed_matches

,id,country_name,league_name,season,date,home_team,away_team,home_team_goal,away_team_goal
0,4390,England,England Premier League,2015/2016,2015-08-08 00:00:00,Bournemouth,Aston Villa,0,1
1,4391,England,England Premier League,2015/2016,2015-08-08 00:00:00,Chelsea,Swansea City,2,2
2,4392,England,England Premier League,2015/2016,2015-08-08 00:00:00,Everton,Watford,2,2
3,4393,England,England Premier League,2015/2016,2015-08-08 00:00:00,Leicester City,Sunderland,4,2
4,4394,England,England Premier League,2015/2016,2015-08-08 00:00:00,Manchester United,Tottenham Hotspur,1,0
5,4396,England,England Premier League,2015/2016,2015-08-08 00:00:00,Norwich City,Crystal Palace,1,3
6,4389,England,England Premier League,2015/2016,2015-08-09 00:00:00,Arsenal,West Ham United,0,2
7,4395,England,England Premier League,2015/2016,2015-08-09 00:00:00,Newcastle United,Southampton,2,2
8,4397,England,England Premier League,2015/2016,2015-08-09 00:00:00,Stoke City,Liverpool,0,1
9,4398,England,England Premier League,2015/2016,2015-08-10 00:00:00,West Bromwich Albion,Manchester City,0,3


# 3. Exploring 'CASE WHEN'

In SQL, CASE WHEN statements allow us to create a variety of different summaries and insightful figures. Lets use them to explore the data.

It's natural that, I will only try find data of my favorite club, which in this is Case is "Totenham Hotspur" 🤪

In [7]:
tot = pd.read_sql("""
SELECT m.date, HT.team_long_name AS  home_team, AT.team_long_name AS away_team,
       home_team_goal, away_team_goal,
       CASE WHEN m.home_team_api_id = 8586 and
       m.home_team_goal > m.away_team_goal THEN 'Spurs won'
       WHEN m.home_team_api_id = 8586 and  m.home_team_goal < m.away_team_goal
       THEN 'Spurs lost'
       WHEN m.away_team_api_id = 8586 and m.home_team_goal < m.away_team_goal
       THEN 'Spurs won'
       WHEN m.away_team_api_id = 8586 and m.home_team_goal > m.away_team_goal
       THEN 'Spurs lost' ELSE 'Tie' END AS outcome
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
Order by 1;""", conn)

tot

,date,home_team,away_team,home_team_goal,away_team_goal,outcome
0,2008-08-16 00:00:00,Middlesbrough,Tottenham Hotspur,2,1,Spurs lost
1,2008-08-23 00:00:00,Tottenham Hotspur,Sunderland,1,2,Spurs lost
2,2008-08-31 00:00:00,Chelsea,Tottenham Hotspur,1,1,Tie
3,2008-09-15 00:00:00,Tottenham Hotspur,Aston Villa,1,2,Spurs lost
4,2008-09-21 00:00:00,Tottenham Hotspur,Wigan Athletic,0,0,Tie
...,...,...,...,...,...,...
299,2016-04-18 00:00:00,Stoke City,Tottenham Hotspur,0,4,Spurs won
300,2016-04-25 00:00:00,Tottenham Hotspur,West Bromwich Albion,1,1,Tie
301,2016-05-02 00:00:00,Chelsea,Tottenham Hotspur,2,2,Tie
302,2016-05-08 00:00:00,Tottenham Hotspur,Southampton,1,2,Spurs lost


Now that we have the match outcome in a CASE WHEN statement, we can use it to create easy-to-view information about how Spurs performed each season. Using the GROUP BY clause, let's see how many matches Spurs won, lost, or drew each season.

In other words, we can use the CASE WHEN statement to create a new column that indicates whether Spurs won, lost, or drew each match. Then, we can use the GROUP BY clause to group the matches by season and count the number of matches in each category.

In [8]:
totout = pd.read_sql("""
SELECT m.season,
    CASE WHEN m.home_team_api_id = 8586 and m.home_team_goal > m.away_team_goal
    THEN 'Spurs won'
    WHEN m.home_team_api_id = 8586 and  m.home_team_goal < m.away_team_goal
    THEN 'Spurs lost'
    WHEN m.away_team_api_id = 8586 and m.home_team_goal < m.away_team_goal
    THEN 'Spurs won'
    WHEN m.away_team_api_id = 8586 and m.home_team_goal > m.away_team_goal
    THEN 'Spurs lost' ELSE 'Tie' END AS outcome,
    count (m.date) AS count
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
GROUP BY m.season,outcome;""", conn)

totout

,season,outcome,count
0,2008/2009,Spurs lost,15
1,2008/2009,Spurs won,14
2,2008/2009,Tie,9
3,2009/2010,Spurs lost,10
4,2009/2010,Spurs won,21
5,2009/2010,Tie,7
6,2010/2011,Spurs lost,8
7,2010/2011,Spurs won,16
8,2010/2011,Tie,14
9,2011/2012,Spurs lost,9


This code would also produce the same result, if run in MySQL:

In [ ]:
# SELECT
#  season,
#  COUNT(CASE WHEN match_outcome = 'W' THEN 1 ELSE NULL END) AS wins,
#  COUNT(CASE WHEN match_outcome = 'L' THEN 1 ELSE NULL END) AS losses,
#  COUNT(CASE WHEN match_outcome = 'D' THEN 1 ELSE NULL END) AS draws
# FROM match
# WHERE team_long_name = 'Spurs'
# GROUP BY season;

In addition to match outcomes, football fans also like to see how many goals their team scored each season. To calculate this, we can create another CASE WHEN statement, then wrap it in a SUM function and group it by season.

In other words, we can use the CASE WHEN statement to create a new column that indicates the number of goals scored by Spurs in each match. Then, we can use the SUM function to calculate the total number of goals scored by Spurs each season. Finally, we can use the GROUP BY clause to group the matches by season and return the total number of goals scored in each season.

In [9]:
GoalsBySeason = pd.read_sql("""
SELECT m.season,
  SUM(CASE WHEN m.home_team_api_id = 8586  THEN home_team_goal END)
  AS home_goals,
  SUM(CASE WHEN m.away_team_api_id = 8586  THEN away_team_goal END) AS away_goals
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
GROUP BY m.season;""", conn)

GoalsBySeason

,season,home_goals,away_goals
0,2008/2009,21,24
1,2009/2010,40,27
2,2010/2011,30,25
3,2011/2012,39,27
4,2012/2013,29,37
5,2013/2014,30,25
6,2014/2015,31,27
7,2015/2016,35,34


MySQL query that we could use to do this:

In [ ]:
# SELECT season,
#  SUM(CASE WHEN team_long_name = 'Spurs' THEN goals_scored ELSE 0 END)
#  AS goals_scored
# FROM match
# GROUP BY season;

We can easily modify the previous query to calculate the average number of home and away goals scored by Spurs each season. To do this, we simply replace the SUM() function with the AVG() function. We also use the ROUND() function to round the results to two decimal places.

In [10]:
AvgGoalsBySeason = pd.read_sql("""
SELECT m.season, ROUND(AVG(CASE WHEN m.home_team_api_id = 8586
      THEN home_team_goal END),2) AS average_home_goals,
      ROUND(AVG(CASE WHEN m.home_team_api_id = 8586
      THEN away_team_goal END),2) AS average_away_goals
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
GROUP BY m.season;""", conn)

AvgGoalsBySeason

,season,average_home_goals,average_away_goals
0,2008/2009,1.11,0.53
1,2009/2010,2.11,0.63
2,2010/2011,1.58,1.00
3,2011/2012,2.05,0.89
4,2012/2013,1.53,0.95
5,2013/2014,1.58,1.21
6,2014/2015,1.63,1.26
7,2015/2016,1.84,0.79


Modified MySQL query that we could use to calculate the average number of home and away goals scored by Spurs each season:

In [ ]:
# SELECT season,
#  ROUND(AVG(CASE WHEN team_long_name = 'Spurs' AND home_away = 'H'
#            THEN goals_scored ELSE 0 END), 2) AS average_home_goals,
#  ROUND(AVG(CASE WHEN team_long_name = 'Spurs' AND home_away = 'A'
#            THEN goals_scored ELSE 0 END), 2) AS average_away_goals
# FROM match
# GROUP BY season;

Now let's create a different CASE WHEN showing what percentage of home and away matches Spurs won.

In [11]:
AvgGoalsBySeasonPCT = pd.read_sql(
"""
SELECT m.season, ROUND(AVG(CASE WHEN m.home_team_api_id = 8586
       AND home_team_goal > away_team_goal THEN 1
       WHEN m.home_team_api_id = 8586 AND home_team_goal < away_team_goal
       THEN 0 END),2) AS  pct_homewins,
       ROUND(AVG(CASE WHEN m.away_team_api_id = 8586
       AND away_team_goal > home_team_goal THEN 1
       WHEN m.home_team_api_id = 8586 AND away_team_goal < home_team_goal
       THEN 0 END),2) AS  pct_awaywins
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
GROUP BY m.season;""", conn)

AvgGoalsBySeasonPCT

,season,pct_homewins,pct_awaywins
0,2008/2009,0.71,0.29
1,2009/2010,0.82,0.33
2,2010/2011,0.90,0.44
3,2011/2012,0.81,0.35
4,2012/2013,0.79,0.48
5,2013/2014,0.69,0.48
6,2014/2015,0.63,0.47
7,2015/2016,0.77,0.47


**Comparing different leagues: across different countries/leagues, what percentage of matches end in a draw?**

The European Soccer Database allows us to compare how football is played in different countries and leagues. To see how this compares, let's compare the percentage of matches that ended in a draw across different leagues for the 2013/14 and 2014/15 seasons.

To do this, we can write a SQL query that joins the country and match tables. We then use a CASE WHEN statement to count the number of draws in each season for each country. Finally, we use the AVG function to calculate the average percentage of draws for each country.

In [12]:
perdraw = pd.read_sql("""
SELECT c.name AS country,
       ROUND(AVG(CASE WHEN m.season='2013/2014'
       AND m.home_team_goal = m.away_team_goal THEN 1
       WHEN m.season='2013/2014' AND m.home_team_goal != m.away_team_goal THEN 0
       END),2) AS pct_ties_2013_2014,
       ROUND(AVG(CASE WHEN m.season='2014/2015'
       AND m.home_team_goal = m.away_team_goal THEN 1
       WHEN m.season='2014/2015' AND m.home_team_goal != m.away_team_goal THEN 0
       END),2) AS pct_ties_2014_2015
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
GROUP BY country
;""", conn)

perdraw

,country,pct_ties_2013_2014,pct_ties_2014_2015
0,Belgium,0.17,0.25
1,England,0.21,0.24
2,France,0.28,0.23
3,Germany,0.21,0.27
4,Italy,0.24,0.32
5,Netherlands,0.27,0.24
6,Poland,0.30,0.28
7,Portugal,0.25,0.28
8,Scotland,0.22,0.19
9,Spain,0.23,0.24


Thus, In the 2014/2015 season, Italy had the highest proportion of draws (almost 1/3) and Scotland had the lowest (about 1/5). Following on from this, what league had the most number of goals, averaging out across all the matches within that league?

In [13]:
leagues_by_avg_goal = pd.read_sql("""
SELECT l.name AS league,
       ROUND(AVG(m.home_team_goal + m.away_team_goal),2) AS avg_goals
FROM league as L
LEFT JOIN match AS m ON l.id = m.country_id
WHERE m.season = '2014/2015'
GROUP BY league
;""", conn)

leagues_by_avg_goal

,league,avg_goals
0,Belgium Jupiler League,2.78
1,England Premier League,2.57
2,France Ligue 1,2.49
3,Germany 1. Bundesliga,2.75
4,Italy Serie A,2.69
5,Netherlands Eredivisie,3.08
6,Poland Ekstraklasa,2.62
7,Portugal Liga ZON Sagres,2.49
8,Scotland Premier League,2.57
9,Spain LIGA BBVA,2.66


### We can see that the Netherlands had the highest average number of goals with 3.08. France and Portugal both had roughly the lowest number of average goals with 2.49.

# 4. Exploring SUB_Queries:

Lets start by working out what was the average number of goals,across all leagues, during the 2014/15 season.

In [14]:
avg_goal1415 = pd.read_sql("""
SELECT ROUND(AVG(m.home_team_goal + m.away_team_goal),2) AS avg_goals
FROM match AS m
WHERE m.season = '2014/2015';""", conn)

avg_goal1415

,avg_goals
0,2.68


### Sub Query within WHERE:

So the average number of goals was 2.68. Lets use this result, as a query within a query, right in the WHERE clause of a bigger query, to find the matches in the French league, where more goals were scored than this average.

In [15]:
Higher_than_avg_goals1415 = pd.read_sql("""
SELECT HT.team_long_name AS  home_team, AT.team_long_name AS away_team,
       m.home_team_goal, m.away_team_goal
FROM match AS m
LEFT JOIN League as l ON l.id = m.country_id
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.season = '2014/2015' AND l.name = 'France Ligue 1'
      AND (m.home_team_goal + m.away_team_goal) >
      (SELECT
      AVG(m.home_team_goal + m.away_team_goal) AS avg_goals
      FROM match AS m
WHERE m.season = '2014/2015');""", conn)

Higher_than_avg_goals1415

,home_team,away_team,home_team_goal,away_team_goal
0,Stade de Reims,Paris Saint-Germain,2,2
1,SC Bastia,Olympique de Marseille,3,3
2,Évian Thonon Gaillard FC,SM Caen,0,3
3,AS Monaco,FC Lorient,1,2
4,OGC Nice,Toulouse FC,3,2
...,...,...,...,...
165,FC Metz,Stade de Reims,3,0
166,Montpellier Hérault SC,En Avant de Guingamp,2,1
167,SM Caen,Olympique de Marseille,1,2
168,Évian Thonon Gaillard FC,FC Metz,3,0


Since we are using historical data here , so that average number of goals for 2014/15 won't change. However imagine the use of this on a sales database which is refreshed daily. You could run this type of query and always see the latest information on which products or shops were beating the average in terms of sales.

### Sub Query within FROM:

We can also use a sub query in the FROM statement. You could think of it this as making a query or temporary table and then querying it again to select only what you want. In the below example the sub query pulls all the matches in the database, but then the WHERE statement selects only those matches where more than 10 goals were scored.

In [16]:
matchesabove10 = pd.read_sql("""
SELECT country,date,total_goals
FROM (SELECT  c.name as country, m.date, m.home_team_goal, m.away_team_goal,
     (m.home_team_goal + m.away_team_goal) as total_goals
     FROM match AS m
     LEFT JOIN country as c ON m.country_id = c.id) AS subquery
WHERE
total_goals >=10
;""", conn)

matchesabove10

,country,date,total_goals
0,England,2009-11-22 00:00:00,10
1,England,2011-08-28 00:00:00,10
2,England,2012-12-29 00:00:00,10
3,England,2013-05-19 00:00:00,10
4,France,2009-11-08 00:00:00,10
5,Germany,2013-03-30 00:00:00,11
6,Netherlands,2010-10-24 00:00:00,10
7,Netherlands,2011-11-06 00:00:00,10
8,Scotland,2010-05-05 00:00:00,12
9,Spain,2013-10-30 00:00:00,10


### Finally, Sub Query within SELECT:

Below we get the date and total goals scored in every single match in the 2014/15 season. Then with a sub query in the select statement we can calculate the average total of goals scored in every match in the whole dataset. This allows us to compare that average to what was scored in the individual matches.

In [17]:
TotalGoalsVAvG = pd.read_sql("""
SELECT date, (m.home_team_goal + m.away_team_goal) AS total_goals,
       (SELECT ROUND(AVG(m.home_team_goal + m.away_team_goal),2)
        FROM match AS m
        WHERE season = '2014/2015') AS overall_avg
FROM match m
WHERE m.season = '2014/2015'
;""", conn)

TotalGoalsVAvG

,date,total_goals,overall_avg
0,2014-07-25 00:00:00,3,2.68
1,2014-07-26 00:00:00,2,2.68
2,2014-07-26 00:00:00,0,2.68
3,2014-07-26 00:00:00,2,2.68
4,2014-07-26 00:00:00,1,2.68
...,...,...,...
3320,2014-09-23 00:00:00,4,2.68
3321,2014-09-23 00:00:00,5,2.68
3322,2014-09-24 00:00:00,3,2.68
3323,2014-09-24 00:00:00,0,2.68


# 5. Common Table Expressionss (CTEs):

Let's create a simple CTE:

In [18]:
simpleCTE = pd.read_sql("""
WITH CTE AS (
  SELECT *
  FROM match )
SELECT
date,
home_team_goal
FROM CTE
;""", conn)

simpleCTE

,date,home_team_goal
0,2008-08-17 00:00:00,1
1,2008-08-16 00:00:00,0
2,2008-08-16 00:00:00,0
3,2008-08-17 00:00:00,5
4,2008-08-16 00:00:00,1
...,...,...
25974,2015-09-22 00:00:00,1
25975,2015-09-23 00:00:00,1
25976,2015-09-23 00:00:00,2
25977,2015-09-22 00:00:00,0


Here the CTE selects everything in the match table and declares that with the name 'cte'. Then we query the date and number of home team goals from that CTE. CTEs are normally easier to read and understand than sub queries. If you are writing a query with multiple sub queries it might be worth reformatting it into multiple WTH statements. As an example, lets first create a CTE which pulls all the matches where 10 or more goals were scored. Then lets join that information onto the league table, so we get a count of how matches have occurred in each league.

In [19]:
HighScoreMatchesByLeague = pd.read_sql("""
WITH big_game AS (
  SELECT league_id, id
  FROM match
  WHERE home_team_goal + away_team_goal >= 10)
SELECT l.name as league, COUNT(big_game.id) as High_Score_Matches
FROM league as l
LEFT JOIN big_game ON l.id = big_game.league_id
GROUP BY league
;""", conn)

HighScoreMatchesByLeague

,league,High_Score_Matches
0,Belgium Jupiler League,0
1,England Premier League,4
2,France Ligue 1,1
3,Germany 1. Bundesliga,1
4,Italy Serie A,0
5,Netherlands Eredivisie,2
6,Poland Ekstraklasa,0
7,Portugal Liga ZON Sagres,0
8,Scotland Premier League,1
9,Spain LIGA BBVA,5


There are leagues who never had a single match where 10 or more goals were scored. At the other extreme, Spain's LIGA BBVA had the highest number of matches (5) where this happened.

Let's build a query that sums up the total number of goals that were scored in matches and then joining that information to an outer query that sums up the information by a dimension only available outside of the CTE, for example league name.

In [20]:
AvgGoalsByleague = pd.read_sql("""
With match_list as (
  SELECT country_id, (home_team_goal + away_team_goal) AS goals
  FROM match
)
SELECT name, ROUND(AVG (goals),2) as Average_Goals
FROM league
LEFT JOIN match_list ON league.id = match_list.country_id
Group by name
;""", conn)

AvgGoalsByleague

,name,Average_Goals
0,Belgium Jupiler League,2.80
1,England Premier League,2.71
2,France Ligue 1,2.44
3,Germany 1. Bundesliga,2.90
4,Italy Serie A,2.62
5,Netherlands Eredivisie,3.08
6,Poland Ekstraklasa,2.43
7,Portugal Liga ZON Sagres,2.53
8,Scotland Premier League,2.63
9,Spain LIGA BBVA,2.77


When working with a complex query, WITH statements can help breakdown the information required into manageable sections and make the query easier to understand when you return to it later.

# 6. Finally - Window Functions:

We created a query earlier that showed the total goals scored in every single match and also, in a sub query, calculated the average number of goals over the season.

SELECT date, (m.home_team_goal + m.away_team_goal) AS total_goals, (SELECT ROUND(AVG(m.home_team_goal + m.away_team_goal),2) FROM match AS m WHERE season = '2014/2015') AS overall_avg FROM match m WHERE m.season = '2014/2015'

We can also create the same outcome, dropping the sub query, and using an OVER clause instead.

In [21]:
Goals = pd.read_sql("""
SELECT date, (m.home_team_goal +m.away_team_goal) AS total_goals,
       ROUND(AVG(home_team_goal + away_team_goal) OVER (),2) AS overall_avg
FROM Match m
WHERE season = '2014/2015';""", conn)

Goals

,date,total_goals,overall_avg
0,2014-07-25 00:00:00,3,2.68
1,2014-07-26 00:00:00,2,2.68
2,2014-07-26 00:00:00,0,2.68
3,2014-07-26 00:00:00,2,2.68
4,2014-07-26 00:00:00,1,2.68
...,...,...,...
3320,2014-09-23 00:00:00,4,2.68
3321,2014-09-23 00:00:00,5,2.68
3322,2014-09-24 00:00:00,3,2.68
3323,2014-09-24 00:00:00,0,2.68


Lets now use an OVER clause, combined with a RANK function, which will rank the results over our result set. First lets write a simple query that finds the average number of total goals scored per match and groups it by league.

In [22]:
leagues_by_avgG = pd.read_sql("""
SELECT League.name AS league_name,
       ROUND(AVG(home_team_goal + away_team_goal),2) as avg_goals
FROM Match
LEFT JOIN League ON League.id = Match.league_id
GROUP BY league_name
;""", conn)

leagues_by_avgG

,league_name,avg_goals
0,Belgium Jupiler League,2.80
1,England Premier League,2.71
2,France Ligue 1,2.44
3,Germany 1. Bundesliga,2.90
4,Italy Serie A,2.62
5,Netherlands Eredivisie,3.08
6,Poland Ekstraklasa,2.43
7,Portugal Liga ZON Sagres,2.53
8,Scotland Premier League,2.63
9,Spain LIGA BBVA,2.77


The Netherlands Eredivisie had the highest average number of total goals per match out of all the leagues. To see how the leagues rank against each other on this, we could order the query results by this figure. However, we may also want to keep the leagues in their alphabetical order. To still see this rank, we can use a RANK() function, combined with an OVER() clause.

The RANK() function ranks the rows in a table based on the value of a specified column. The OVER() clause specifies the scope of the ranking. In this case, we would want to rank the leagues within each country.

In [23]:
leagues_by_avgGRank = pd.read_sql("""
SELECT League.name AS league_name,
       ROUND(AVG(home_team_goal + away_team_goal),2) as avg_goals,
       RANK () OVER (ORDER BY  AVG (home_team_goal + away_team_goal)DESC)
       AS league_rank
FROM Match
LEFT JOIN League ON League.id = Match.league_id
GROUP BY league_name
ORDER BY League_name;""", conn)

leagues_by_avgGRank

,league_name,avg_goals,league_rank
0,Belgium Jupiler League,2.80,4
1,England Premier League,2.71,6
2,France Ligue 1,2.44,10
3,Germany 1. Bundesliga,2.90,3
4,Italy Serie A,2.62,8
5,Netherlands Eredivisie,3.08,1
6,Poland Ekstraklasa,2.43,11
7,Portugal Liga ZON Sagres,2.53,9
8,Scotland Premier League,2.63,7
9,Spain LIGA BBVA,2.77,5


Here we use the RANK function, tell it to rank over the entire set of results, and to rank by the total average goals (descending). Keep in mind that the OVER clause isn't running directly on the tables in the database, but rather on the results set you have generated.

Next, we have the principle of the OVER clause, lets expand what we can do by combining it with the PARTITION BY clause. PARTITION BY allows us to calculate seperate values for different categories. Rather than having to calculate totals for different categories and group them into columns, we can present them in the same column.

In [24]:
leagues_by_season = pd.read_sql("""
SELECT Match.date, League.name AS league_name, season,
       home_team_goal + away_team_goal as Total_Goals,
       ROUND(AVG(home_team_goal + away_team_goal) OVER (partition by season),2)
       AS season_avg
FROM Match
LEFT JOIN League ON League.id = Match.league_id;""", conn)

leagues_by_season

,date,league_name,season,Total_Goals,season_avg
0,2008-08-17 00:00:00,Belgium Jupiler League,2008/2009,2,2.61
1,2008-08-16 00:00:00,Belgium Jupiler League,2008/2009,0,2.61
2,2008-08-16 00:00:00,Belgium Jupiler League,2008/2009,3,2.61
3,2008-08-17 00:00:00,Belgium Jupiler League,2008/2009,5,2.61
4,2008-08-16 00:00:00,Belgium Jupiler League,2008/2009,4,2.61
...,...,...,...,...,...
25974,2015-09-22 00:00:00,Switzerland Super League,2015/2016,1,2.75
25975,2015-09-23 00:00:00,Switzerland Super League,2015/2016,3,2.75
25976,2015-09-23 00:00:00,Switzerland Super League,2015/2016,2,2.75
25977,2015-09-22 00:00:00,Switzerland Super League,2015/2016,0,2.75


We can then take this even further by adding more fields to the partition by clause. In this case we will add league to the existing partition and so now we will be calculating the average number of goals within each season, within each league.

In [25]:
leagues_by_season_leag = pd.read_sql("""
SELECT Match.date, League.name AS league_name, season,
       (home_team_goal + away_team_goal) as Total_Goals,
       AVG (home_team_goal + away_team_goal)
       OVER (partition by season,League.name) AS season_league_avg
FROM Match
LEFT JOIN League ON League.id = Match.league_id;""", conn)

leagues_by_season_leag

,date,league_name,season,Total_Goals,season_league_avg
0,2008-08-17 00:00:00,Belgium Jupiler League,2008/2009,2,2.794118
1,2008-08-16 00:00:00,Belgium Jupiler League,2008/2009,0,2.794118
2,2008-08-16 00:00:00,Belgium Jupiler League,2008/2009,3,2.794118
3,2008-08-17 00:00:00,Belgium Jupiler League,2008/2009,5,2.794118
4,2008-08-16 00:00:00,Belgium Jupiler League,2008/2009,4,2.794118
...,...,...,...,...,...
25974,2015-09-22 00:00:00,Switzerland Super League,2015/2016,1,3.144444
25975,2015-09-23 00:00:00,Switzerland Super League,2015/2016,3,3.144444
25976,2015-09-23 00:00:00,Switzerland Super League,2015/2016,2,3.144444
25977,2015-09-22 00:00:00,Switzerland Super League,2015/2016,0,3.144444


Perhaps a football fan like me, would like to look at each match their team has played every season and have the average number of home and away goals their team scored on average within that season and month. Lets do this for Spurs again.

In [26]:
leagues_by_season_month_spurs = pd.read_sql("""
SELECT Match.date, strftime('%m', Match.date) as Month, season,
       HT.team_long_name AS  home_team, AT.team_long_name AS away_team,
       home_team_goal, away_team_goal,
       CASE WHEN  Match.home_team_api_id = 8586 THEN 'home' ELSE 'away'
       END AS Home_Or_Away,
       AVG (home_team_goal) OVER (partition by season,strftime('%m',Match.date))
       AS season_month_home_avg,
       AVG (away_team_goal) OVER (partition by season,strftime('%m', Match.date))
       AS season_month_away_avg
FROM Match
LEFT JOIN League ON League.id = Match.league_id
LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
WHERE Match.home_team_api_id = 8586 or Match.away_team_api_id = 8586
;""", conn)

leagues_by_season_month_spurs

,date,Month,season,home_team,away_team,home_team_goal,away_team_goal,Home_Or_Away,season_month_home_avg,season_month_away_avg
0,2009-01-11 00:00:00,01,2008/2009,Wigan Athletic,Tottenham Hotspur,1,0,away,2.0,1.0
1,2009-01-18 00:00:00,01,2008/2009,Tottenham Hotspur,Portsmouth,1,1,home,2.0,1.0
2,2009-01-27 00:00:00,01,2008/2009,Tottenham Hotspur,Stoke City,3,1,home,2.0,1.0
3,2009-01-31 00:00:00,01,2008/2009,Bolton Wanderers,Tottenham Hotspur,3,2,away,2.0,1.0
4,2009-02-08 00:00:00,02,2008/2009,Tottenham Hotspur,Arsenal,0,0,home,0.5,1.0
...,...,...,...,...,...,...,...,...,...,...
299,2015-12-05 00:00:00,12,2015/2016,West Bromwich Albion,Tottenham Hotspur,1,1,away,1.2,1.4
300,2015-12-13 00:00:00,12,2015/2016,Tottenham Hotspur,Newcastle United,1,2,home,1.2,1.4
301,2015-12-19 00:00:00,12,2015/2016,Southampton,Tottenham Hotspur,0,2,away,1.2,1.4
302,2015-12-26 00:00:00,12,2015/2016,Tottenham Hotspur,Norwich City,3,0,home,1.2,1.4


As you can see Spurs' average number of home goals in the month of Jan in the 08/09 season was 2.0. Then when we move into the month of Feb in the same season , the average changes to 0.5. Also notice that the function strftime is used to extract the month from the date field, which then gets used in the partition by.

# Conclusion:

The SQL techniques of CASE WHEN statements, sub queries, WITH statements and window functions provide a flexible range of approaches to exploring data and generating both deep analysis and summary findings.